In [1]:
import pandas as pd
import numpy as np

In [2]:
df_tra=pd.read_csv("/kaggle/input/creditcard-fraud-detection/transactions.csv")
df_cc=pd.read_csv("/kaggle/input/creditcard-fraud-detection/cc_info.csv")

In [3]:
df_tra.isnull().sum()

credit_card                  0
date                         0
transaction_dollar_amount    0
Long                         0
Lat                          0
dtype: int64

In [4]:
df_tra["date"]=pd.to_datetime(df_tra["date"])
df_tra["Year"]=df_tra["date"].dt.year
df_tra["Month"]=df_tra["date"].dt.month
df_tra["Day"]=df_tra["date"].dt.day
df_tra["Hour"]=df_tra["date"].dt.hour
df_tra["Minute"]=df_tra["date"].dt.minute
df_tra["Second"]=df_tra["date"].dt.second

In [5]:
df_tra2=df_tra.drop(columns=["date"])

In [6]:
df_tra2

,credit_card,transaction_dollar_amount,Long,Lat,Year,Month,Day,Hour,Minute,Second
0,1003715054175576,43.78,-80.174132,40.267370,2015,9,11,0,32,40
1,1003715054175576,103.15,-80.194240,40.180114,2015,10,24,22,23,8
2,1003715054175576,48.55,-80.211033,40.313004,2015,10,26,18,19,36
3,1003715054175576,136.18,-80.174138,40.290895,2015,10,22,19,41,10
4,1003715054175576,71.82,-80.238720,40.166719,2015,10,26,20,8,22
...,...,...,...,...,...,...,...,...,...,...
294583,9999757432802760,127.23,-82.360952,32.978497,2015,9,10,19,43,33
294584,9999757432802760,84.90,-82.322721,33.061071,2015,8,6,21,0,13
294585,9999757432802760,77.54,-82.442130,32.949983,2015,9,22,16,15,47
294586,9999757432802760,144.05,-82.398587,32.976162,2015,8,27,18,8,24


# # Deep learning autoencoder

In [15]:
from keras.layers import Dense,LSTM,SimpleRNN,GRU,Flatten,BatchNormalization,TimeDistributed,Input,LeakyReLU,Dropout
from keras.models import Model
from keras.initializers import HeNormal
from keras.regularizers import l2
from keras.callbacks import EarlyStopping

In [8]:
early_stopping=EarlyStopping(
monitor='val_loss',
patience=5,
verbose=1)

In [9]:
df_tra2_values=df_tra2.values
df_tra2_reshaped=np.repeat(df_tra2_values[:,np.newaxis,:],60,axis=1)

In [10]:
df_tra2.shape

(294588, 10)

In [11]:
df_tra2_reshaped.shape

(294588, 60, 10)

In [19]:
inputs=Input(shape=(60,10))
e=SimpleRNN(300,activation=None,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01),return_sequences=True)(inputs)
e=LeakyReLU(0.8)(e)
e=Dropout(0.5)(e)
e=BatchNormalization()(e)
e=SimpleRNN(200,activation=None,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01),return_sequences=True)(e)
e=LeakyReLU(0.8)(e)
e=BatchNormalization()(e)

e=SimpleRNN(100,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01),return_sequences=True)(e)
# e=LeakyReLU(0.8)(e)
e=Dropout(0.5)(e)
e=BatchNormalization()(e)

e=SimpleRNN(50,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01),return_sequences=True)(e)
# e=LeakyReLU(0.8)(e)

d=SimpleRNN(50,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01),return_sequences=True)(e)
# d=LeakyReLU(0.8)(d)
d=BatchNormalization()(d)

d=Dropout(0.5)(d)
d=SimpleRNN(100,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01),return_sequences=True)(d)
# d=LeakyReLU(0.8)(d)
d=SimpleRNN(200,activation=None,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01),return_sequences=True)(d)
d=LeakyReLU(0.8)(d)
d=BatchNormalization()(d)

d=Dropout(0.5)(d)
d=SimpleRNN(300,activation=None,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01),return_sequences=True)(d)
d=LeakyReLU(0.8)(d)
d=BatchNormalization()(d)

outputs=TimeDistributed(Dense(10,activation='sigmoid'))(d)
model=Model(inputs,outputs)
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(df_tra2_reshaped,df_tra2_reshaped,epochs=32,batch_size=32,validation_split=0.2,callbacks=[early_stopping])

Epoch 1/32
7365/7365 ━━━━━━━━━━━━━━━━━━━━ 381s 48ms/step - accuracy: 0.0151 - loss: nan - val_accuracy: 0.0000e+00 - val_loss: nan
Epoch 2/32
7299/7365 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - accuracy: 0.0000e+00 - loss: nan

In [ ]:
predictions = model.predict(df_tra2_reshaped)
reconstruction_error = np.mean(np.square(df_tra2_reshaped - predictions), axis=1)  
threshold = np.mean(reconstruction_error) + 2 * np.std(reconstruction_error)
labels = np.where(reconstruction_error > threshold, 0, 1)
# 0  for normal data

accuracy = np.mean(labels == 0) * 100 
print(f"Accuracy: {accuracy:.2f}%")
